In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import anndata as ad
import decoupler as dc
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib.colors import Normalize, to_hex
from matplotlib.patches import Patch


/home/daniele/miniconda3/envs/scanpy/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# read the data

In [2]:
mouse_all = ad.read_zarr('/mnt/ssd/atlases/Mouse_Atlas_Harmonised.zarr')
human_all = ad.read_zarr('/mnt/ssd/atlases/Human_Atlas_Harmonised.zarr')

In [4]:
hallmark_h = dc.op.hallmark(organism='human')

In [5]:
hm_dict = {}
for hm in hallmark_h.source.unique():
    hm_dict[hm] = hallmark_h[hallmark_h.source==hm].target.unique().tolist()

In [7]:
mouse_all.var["h"] = [g.upper() for g in mouse_all.var_names]

In [8]:
mouse_all.var_names = mouse_all.var["h"]


In [10]:
intersect = list(set(mouse_all.var_names).intersection(set(human_all.var_names)))

In [13]:
mouse_all = mouse_all[:, intersect].copy()
human_all = human_all[:, intersect].copy()

In [14]:
mask_mouse = mouse_all.obs["Level_4"].astype("string").str.contains(
    "Malignant", regex=False, na=False
).to_numpy(dtype=bool)

mask_human = human_all.obs["Level_4"].astype("string").str.contains(
    "Malignant", regex=False, na=False
).to_numpy(dtype=bool)

In [15]:
mouse_mal = mouse_all[mask_mouse].copy()
human_mal = human_all[mask_human].copy()


In [ ]:
mouse_mal = mouse.copy()
human_mal = human.copy()

# pathway overlap

In [16]:
from typing import Dict, List, Literal, Optional
try:
    from anndata import AnnData
except ImportError as e:
    raise ImportError("This function requires `anndata`. Install via `pip install anndata`.") from e

Correlation = Literal["pearson", "spearman"]

def gene_set_coexpression_from_anndatas(
    adata_true: AnnData,
    adata_recon: AnnData,
    geneset_dict: Dict[str, List[str]],
    *,
    overlap_threshold: int = 5,
    min_cells: int = 20,
    threshold: float = 0.0,
    correlation: Correlation = "pearson",
) -> Dict[str, float]:
    """
    Compute gene-set coexpression similarity between 'true' and 'reconstructed' AnnData objects.

    Parameters
    ----------
    adata_true, adata_recon : AnnData
        AnnData with X shaped (n_cells, n_genes). Gene names must be in .var_names.
    geneset_dict : dict[str, list[str]]
        Mapping of gene-set name -> list of gene symbols (matching .var_names).
    overlap_threshold : int
        Minimum overlapping (and expressed) genes required to score a gene set.
    min_cells : int
        A gene counts as "expressed" if it's non-zero in at least this many cells
        (required in BOTH datasets).
    threshold : float
        Pairs with |corr_true| < threshold are ignored (set to NaN) when averaging.
    correlation : {"pearson","spearman"}
        Correlation measure used for coexpression.

    Returns
    -------
    dict
        {geneset_name: score, ..., "average": mean_of_all_set_scores}
        where score = mean over upper triangle of:
            1 - |C_true - C_recon| / 2
        within that gene set.
    """
    # --- helpers --------------------------------------------------------------
    def _to_dense_float(M):
        import scipy.sparse as sp
        return M.toarray().astype(float) if sp.issparse(M) else M.astype(float)

    def _rank_cols(M: np.ndarray) -> np.ndarray:
        # average ranks for ties, column-wise
        R = np.empty_like(M, dtype=float)
        for j in range(M.shape[1]):
            x = M[:, j]
            order = np.argsort(x, kind="mergesort")
            ranks = np.empty_like(order, dtype=float)
            ranks[order] = np.arange(1, len(x) + 1, dtype=float)
            vals, inv = np.unique(x, return_inverse=True)
            for v in range(len(vals)):
                mask = (inv == v)
                if mask.any():
                    avg = ranks[mask].mean()
                    ranks[mask] = avg
            R[:, j] = ranks
        return R

    def _corr_mat(M: np.ndarray, kind: Correlation) -> np.ndarray:
        if M.shape[1] <= 1:
            return np.ones((1, 1), dtype=float)
        if kind == "pearson":
            return np.corrcoef(M, rowvar=False)
        elif kind == "spearman":
            R = _rank_cols(M)
            return np.corrcoef(R, rowvar=False)
        else:
            raise ValueError("correlation must be 'pearson' or 'spearman'")

    # --- align genes by name --------------------------------------------------
    genes_true = np.array(adata_true.var_names, dtype=str)
    genes_recon = np.array(adata_recon.var_names, dtype=str)
    shared = np.intersect1d(genes_true, genes_recon, assume_unique=False)

    if shared.size == 0:
        return {"average": np.nan}

    # index maps
    idx_true = {g: i for i, g in enumerate(genes_true)}
    idx_recon = {g: i for i, g in enumerate(genes_recon)}
    t_cols = np.array([idx_true[g] for g in shared], dtype=int)
    r_cols = np.array([idx_recon[g] for g in shared], dtype=int)

    X_t = _to_dense_float(adata_true.X[:, t_cols])
    X_r = _to_dense_float(adata_recon.X[:, r_cols])
    shared_names = shared  # keeps same order as columns above

    # --- filter to expressed genes in BOTH matrices ---------------------------
    expr_t = (X_t != 0).sum(axis=0) >= min_cells
    expr_r = (X_r != 0).sum(axis=0) >= min_cells
    keep = expr_t & expr_r

    if not np.any(keep):
        return {"average": np.nan}

    X_t = X_t[:, keep]
    X_r = X_r[:, keep]
    kept_names = shared_names[keep]
    name_to_kept = {g: i for i, g in enumerate(kept_names)}

    # --- score each gene set --------------------------------------------------
    results: Dict[str, float] = {}

    for gset_name, genes in geneset_dict.items():
        idxs = [name_to_kept[g] for g in genes if g in name_to_kept]
        idxs = sorted(set(idxs))
        if len(idxs) < max(2, overlap_threshold):
            continue

        A = X_t[:, idxs]
        B = X_r[:, idxs]

        C_true = _corr_mat(A, correlation)
        C_recon = _corr_mat(B, correlation)

        # ignore low-magnitude pairs per true corr
        mask_low = np.abs(C_true) < threshold
        diff = np.abs(C_true - C_recon)
        diff[mask_low] = np.nan

        sim = 1.0 - diff / 2.0
        np.fill_diagonal(sim, np.nan)

        tri = sim[np.triu_indices(sim.shape[0], k=1)]
        score = np.nanmean(tri) if tri.size else np.nan
        results[gset_name] = float(score)

    results["average"] = float(np.nanmean(list(results.values()))) if results else np.nan
    return results


In [17]:
results = {}

In [18]:
models = ['orthotopic', 'endogenous']
for ct in mouse_mal.obs["Level_4"].unique():
    input_m = mouse_mal[mouse_mal.obs.Level_4 == ct] 
    input_h = human_mal[human_mal.obs.Level_4 == ct]
    for md in models:
        ad_m = input_m[input_m.obs.Model == md] 
        results[f"{ct}_{md}"] = gene_set_coexpression_from_anndatas(ad_m, input_h, hm_dict)

In [20]:
from collections import defaultdict
pathway_table = defaultdict(dict)

for ct_model, scores in results.items():
    for pathway, val in scores.items():
        if pathway == "average":
            continue
        pathway_table[pathway][ct_model] = val

df_pathways_wide = pd.DataFrame.from_dict(pathway_table, orient="index").sort_index()


def split_ct_model(col):
    ct, md = col.rsplit("_", 1)
    return ct, md

multi_cols = [split_ct_model(c) for c in df_pathways_wide.columns]
df_pathways_wide.columns = pd.MultiIndex.from_tuples(multi_cols, names=["cell_type", "model"])

model_order = ["orthotopic", "endogenous"]
df_pathways_wide = df_pathways_wide.reindex(columns=pd.MultiIndex.from_tuples(
    sorted(df_pathways_wide.columns, key=lambda t: (model_order.index(t[1]) if t[1] in model_order else 99, t[0]))
))

In [24]:
df_pathways_wide.to_csv("/home/daniele/Code/github_synced/PDAC/supplementary_data/pathway_correlations_matrix.csv")

In [25]:
df_pathways_wide

,Malignant Cell - Acinar-like,Malignant Cell - Apoptotic,Malignant Cell - EMT,Malignant Cell - Epithelial,Malignant Cell - Highly Invasive,Malignant Cell - Highly Proliferative,Malignant Cell - Hypoxia,Malignant Cell - Mesenchymal,Malignant Cell - Pit Like,Malignant Cell - Senescence,Malignant Cell - Acinar-like,Malignant Cell - Apoptotic,Malignant Cell - EMT,Malignant Cell - Epithelial,Malignant Cell - Highly Invasive,Malignant Cell - Highly Proliferative,Malignant Cell - Hypoxia,Malignant Cell - Mesenchymal,Malignant Cell - Pit Like,Malignant Cell - Senescence
,orthotopic,orthotopic,orthotopic,orthotopic,orthotopic,orthotopic,orthotopic,orthotopic,orthotopic,orthotopic,endogenous,endogenous,endogenous,endogenous,endogenous,endogenous,endogenous,endogenous,endogenous,endogenous
ADIPOGENESIS,0.925444,0.930848,0.926109,0.948790,0.958803,0.960037,0.954407,0.950013,0.945926,0.951798,0.955482,0.954718,0.935065,0.964178,0.940401,0.949022,0.934224,0.964739,0.943070,0.964199
ALLOGRAFT_REJECTION,0.955939,0.958204,0.954770,0.965563,0.958627,0.964517,0.965040,0.962678,0.959024,0.967012,0.962767,0.957822,0.937516,0.964807,0.921591,0.956503,0.908694,0.960961,0.956870,0.966282
ANDROGEN_RESPONSE,0.914413,0.925747,0.927421,0.941560,0.949563,0.946868,0.947037,0.946953,0.933716,0.951147,0.959687,0.956771,0.935925,0.957008,0.931039,0.949277,0.925790,0.964219,0.940344,0.964670
ANGIOGENESIS,0.946029,0.945856,0.944583,0.949746,0.939383,0.947363,0.955688,0.950937,0.937446,0.959337,0.953361,0.949210,0.919272,0.965689,0.918805,0.944935,0.915558,0.949896,0.958040,0.960245
APICAL_JUNCTION,0.946671,0.947245,0.949727,0.960320,0.950639,0.957478,0.960608,0.957833,0.948385,0.963797,0.964763,0.956210,0.927536,0.967601,0.926682,0.948138,0.926182,0.962836,0.949147,0.966689
APICAL_SURFACE,0.949443,0.960372,0.966081,0.960076,0.967648,0.968020,0.972197,0.966715,0.945453,0.973651,0.967483,0.971716,0.937813,0.967386,0.927631,0.954078,0.929512,0.962351,0.951938,0.973699
APOPTOSIS,0.935196,0.943197,0.939462,0.954136,0.954860,0.957564,0.959062,0.950809,0.945992,0.958197,0.957584,0.959550,0.936364,0.961646,0.931617,0.946518,0.928828,0.961382,0.942018,0.961814
BILE_ACID_METABOLISM,0.953473,0.958521,0.965326,0.964286,0.966340,0.970600,0.971787,0.974262,0.963606,0.968487,0.968165,0.969507,0.938482,0.973135,0.948319,0.960445,0.941822,0.976796,0.955918,0.973902
CHOLESTEROL_HOMEOSTASIS,0.920491,0.926889,0.925611,0.945282,0.954041,0.954898,0.950082,0.943631,0.942481,0.948170,0.954217,0.943161,0.923252,0.960173,0.937306,0.948397,0.940208,0.953909,0.949153,0.962728


In [ ]:
results_to_plot = {cell_type: values['average'] for cell_type, values in results.items()}
orthotopic = {}
endogenous = {}

for key, value in results_to_plot.items():
    if key.endswith('_orthotopic'):
        cell_type = key.replace('_orthotopic', '')
        orthotopic[cell_type] = value
    elif key.endswith('_endogenous'):
        cell_type = key.replace('_endogenous', '')
        endogenous[cell_type] = value

In [ ]:
df = pd.DataFrame({
    'cell_type': list(orthotopic.keys()),
    'Orthotopic': [orthotopic[k] for k in orthotopic.keys()],
    'Endogenous': [endogenous[k] for k in orthotopic.keys()]
}).sort_values('Orthotopic', ascending=False)

In [ ]:
x = np.arange(len(df))
bar_width = 0.35

plt.figure(figsize=(18, 6))
plt.bar(x - bar_width/2, df['Orthotopic'], width=bar_width, color='steelblue', label='Orthotopic')
plt.bar(x + bar_width/2, df['Endogenous'], width=bar_width, color='orange', label='Endogenous')

wrapped_labels = ["\n".join(wrap(lbl.replace("Malignant Cell - ", ""), 15)) for lbl in df["cell_type"]]
plt.xticks(x, wrapped_labels, rotation=0, fontsize=9)
plt.ylabel("Average score", fontsize=11)
plt.title("Pathway Averages by Cell Type: Orthotopic vs Endogenous", fontsize=13, pad=10)
plt.ylim(0.8, 1.0)

plt.grid(False)
plt.legend(frameon=False, bbox_to_anchor=(1.02, 1), loc='upper left')

plt.tight_layout(pad=2)
plt.savefig("/mnt/kkf2/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure6/pathway_correlation.png", dpi = 300)
plt.show()


# celltype wassertein

In [ ]:
import ot

In [ ]:
def sinkhorn_divergence(X, Y, reg=0.05, unbalanced=False, tau=1.0):
    a = np.ones(X.shape[0]) / X.shape[0]
    b = np.ones(Y.shape[0]) / Y.shape[0]
    M = ot.dist(X, Y, metric='euclidean') ** 2  # cost matrix
    if unbalanced:
        return ot.unbalanced.sinkhorn_unbalanced2(a, b, M, reg, tau)
    else:
        # Sinkhorn divergence = 2*OT - self terms
        ot_xy = ot.sinkhorn2(a, b, M, reg)
        ot_xx = ot.sinkhorn2(a, a, ot.dist(X, X)**2, reg)
        ot_yy = ot.sinkhorn2(b, b, ot.dist(Y, Y)**2, reg)
        return ot_xy - 0.5*ot_xx - 0.5*ot_yy


In [ ]:
def sinkhorn_w2_gene_space(X, Y, reg=0.01):
    # X, Y: (n, g) z-scored gene matrices; cost = ||x-y||^2
    a = np.ones(X.shape[0]) / X.shape[0]
    b = np.ones(Y.shape[0]) / Y.shape[0]
    M = ot.dist(X, Y, metric='euclidean') ** 2
    # Sinkhorn divergence (unbiased): 2*OT(X,Y) - OT(X,X) - OT(Y,Y)
    ot_xy = ot.sinkhorn2(a, b, M, reg)
    ot_xx = ot.sinkhorn2(a, a, ot.dist(X, X, metric='euclidean')**2, reg)
    ot_yy = ot.sinkhorn2(b, b, ot.dist(Y, Y, metric='euclidean')**2, reg)
    return ot_xy - 0.5*ot_xx - 0.5*ot_yy

def subsample_equal(X, Y, max_cells=2000, rng=0):
    rs = np.random.RandomState(rng)
    nx, ny = X.shape[0], Y.shape[0]
    k = min(max_cells, nx, ny)
    if k < 10:
        return None, None
    ix = rs.choice(nx, size=k, replace=False)
    iy = rs.choice(ny, size=k, replace=False)
    return X[ix], Y[iy]

In [ ]:
cell_types = human_mal.obs['Level_4'].unique().tolist()


In [ ]:
rows = []
for ct in cell_types:
    print(f"processing {ct}")
    X = human_mal[human_mal.obs['Level_4'] == ct].layers['counts']
    X = X.toarray() if hasattr(X, "toarray") else X
    Y = mouse_mal[mouse_mal.obs['Level_4'] == ct].layers['counts']
    Y = Y.toarray() if hasattr(Y, "toarray") else Y

    Xs, Ys = subsample_equal(X, Y, max_cells=2000, rng=0)
    D_sinkhorn = float(sinkhorn_w2_gene_space(Xs, Ys, reg=0.1))
    rows.append((ct, D_sinkhorn))


In [ ]:
rows

# celltype distributions analysis

In [ ]:
def celltype_composition(adata, group_col='Sample_ID', celltype_col='Level_4'):
    ct_counts = (
        adata.obs
        .groupby([group_col, celltype_col])
        .size()
        .unstack(fill_value=0)
    )
    ct_frac = ct_counts.div(ct_counts.sum(axis=1), axis=0)
    return ct_frac

mouse_comp = celltype_composition(mouse)
human_comp = celltype_composition(human)
common_cts = set(mouse_comp.columns).union(human_comp.columns)
mouse_comp = mouse_comp.reindex(columns=common_cts, fill_value=0)
human_comp = human_comp.reindex(columns=common_cts, fill_value=0)

In [ ]:
bulk = sc.read("/mnt/storage/Shrey/PDAC_Downstream/ps_adata_all_cells_bulk_filtered.h5ad")

In [ ]:
def make_sample_id(sid, ds):
    if sid.startswith(ds + "_"):
        return sid[len(ds) + 1:]
    if sid.startswith(ds):
        return sid[len(ds):]
    return sid

bulk.obs["Sample_ID"] = [
    make_sample_id(sid, ds) for sid, ds in zip(bulk_obs["Dataset_ID"], bulk_obs["Dataset"])
]
mapping = {
    'Epi-High': '1',
    'Acinar-Like': '1',
    'Hypoxia/Senescence-High': '2',
    'EMT-Start': '3',
    'Tip':'4',
    'Mes-High':'5'
}
bulk.obs['leiden_cell_comp'] = bulk.obs['Cluster_Names'].map(mapping)

In [ ]:
from scipy.spatial.distance import jensenshannon

dist_mat = pd.DataFrame(
    np.zeros((len(mouse_comp), len(human_comp))),
    index=mouse_comp.index,
    columns=human_comp.index
)

for m in mouse_comp.index:
    for h in human_comp.index:
        dist_mat.loc[m, h] = jensenshannon(mouse_comp.loc[m], human_comp.loc[h])
        
model_map = {k:v for k,v in zip(mouse_all.obs.Sample_ID, mouse_all.obs.Model)}
dist_mat["Model"] = dist_mat.index.map(model_map)
leiden_map = {k:v for k,v in zip(bulk.obs.Sample_ID, bulk.obs.leiden_cell_comp)}
emt_map = {k:v for k,v in zip(bulk.obs.Sample_ID, bulk.obs.mesenchymal_ecm_markers_score)}
leiden_row = pd.Series(dist_mat.columns.map(leiden_map), index=dist_mat.columns, name="leiden")
emt_row = pd.Series(dist_mat.columns.map(emt_map), index=dist_mat.columns, name="EMT")
dist_mat_l = pd.concat([dist_mat, leiden_row.to_frame().T, emt_row.to_frame().T])
dist_mat_l = dist_mat_l.loc[:, ~dist_mat_l.loc["leiden"].isna()]
cols = list(dist_mat_l.columns)
cols.append("Model")
dist_mat = dist_mat.loc[:, cols]
dist_mat_l.lo

In [ ]:
model_colors = {"endogenous": "#1f77b4", "orthotopic": "#ff7f0e"}
row_colors = mouse_meta.loc[mat.index, "Model"].map(model_colors)

unique_leiden = sorted(pd.Series(leiden_series.loc[mat.columns]).astype(str).unique())
leiden_palette = dict(zip(unique_leiden, sns.color_palette("tab10", len(unique_leiden))))
col_colors_leiden = pd.Series(leiden_series.loc[mat.columns].astype(str)).map(leiden_palette)

emt_vals = pd.to_numeric(emt_series.loc[mat.columns], errors="coerce")
norm_emt = Normalize(vmin=0.0, vmax=0.15)
cmap_emt = sns.mpl_palette("viridis", as_cmap=True) 
col_colors_emt = pd.Series(
    [to_hex(cmap_emt(norm_emt(v))) if np.isfinite(v) else "#D3D3D3" for v in emt_vals],
    index=mat.columns,
    name="EMT"
)

col_colors_df = pd.DataFrame({"Leiden": col_colors_leiden, "EMT": col_colors_emt}, index=mat.columns)

g = sns.clustermap(
    mat,
    row_cluster=True,
    col_cluster=True,
    row_colors=row_colors,
    col_colors=col_colors_df,
    cmap="coolwarm",
    vmin=0.25, vmax=0.75,
    figsize=(12, 8),
    cbar_pos=None,
    dendrogram_ratio=(.15, .15),
    colors_ratio=(.03, .03)
)

sm_emt = mpl.cm.ScalarMappable(norm=norm_emt, cmap=cmap_emt)
sm_emt.set_array([])
cax_emt = g.fig.add_axes([.95, 0.25, 0.015, 0.5])  # [left, bottom, width, height]
cbar_emt = g.fig.colorbar(sm_emt, cax=cax_emt, orientation="vertical")
cbar_emt.set_label("EMT score (0–0.3)", fontsize=9)

sm_hm = mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(vmin=0.25, vmax=0.75), cmap="coolwarm")
sm_hm.set_array([])
cax_hm = g.fig.add_axes([1.05, 0.25, 0.015, 0.5])
cbar_hm = g.fig.colorbar(sm_hm, cax=cax_hm, orientation="vertical")
cbar_hm.set_label("Heatmap", fontsize=9)

handles = [Patch(facecolor=c, label=f"Model: {k}") for k, c in model_colors.items()]
handles += [Patch(facecolor=c, label=f"Leiden {k}") for k, c in leiden_palette.items()]
g.fig.legend(handles=handles, loc="center left", bbox_to_anchor=(0.80, 0.5), frameon=False, title="Annotations")

g.ax_heatmap.set_xticklabels([])
g.ax_heatmap.set_yticklabels([])
g.ax_heatmap.tick_params(left=False, bottom=False)
g.ax_heatmap.set_xlabel("")
g.ax_heatmap.set_ylabel("")
plt.savefig("/mnt/kkf2/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure6/cellcomp_corr.png", dpi = 300)

plt.show()


# DGE overlap

In [ ]:
def _sanitize_obsm_varm(a: ad.AnnData):
    # Move any 1-D arrays out of .obsm/.varm
    for attr in ["obsm", "varm"]:
        mapping = getattr(a, attr)
        bad_keys = []
        for k, v in list(mapping.items()):
            # Convert pandas objects to numpy for ndim check
            arr = v.values if hasattr(v, "values") else v
            if getattr(arr, "ndim", None) != 2:
                bad_keys.append(k)
        for k in bad_keys:
            v = mapping[k]
            # Decide target: vectors tied to cells -> .obs, tied to genes -> .var
            if attr == "obsm":    # per-cell
                a.obs[k] = np.asarray(v).ravel()
            else:                 # per-gene
                a.var[k] = np.asarray(v).ravel()
            del mapping[k]

def _align_obsm_keys(a1: ad.AnnData, a2: ad.AnnData):
    common = set(a1.obsm.keys()) & set(a2.obsm.keys())
    for a in (a1, a2):
        for k in list(a.obsm.keys()):
            if k not in common:
                del a.obsm[k]

In [ ]:
_sanitize_obsm_varm(mouse_mal)
_sanitize_obsm_varm(human_mal)
_align_obsm_keys(mouse_mal, human_mal)

In [ ]:
all_together = ad.concat([human_mal, mouse_mal], label='Organism', keys=['human','mouse'])

In [ ]:
all_together

In [ ]:
all_together.obs['Model'] = mouse_mal.obs.Model.astype(str)
all_together.obs['Model'] = all_together.obs['Model'].replace(np.nan, 'human').astype('category')
all_together.obs['Model'].value_counts()

In [ ]:
dge_list = {}
sc.tl.rank_genes_groups(all_together, groups=['endogenous'], reference='human', groupby = 'Model', layer='log_norm')
endog_df = sc.get.rank_genes_groups_df(all_together, group='endogenous')
sc.tl.rank_genes_groups(all_together, groups=['orthotopic'], reference='human', groupby = 'Model', layer='log_norm')
ortho_df = sc.get.rank_genes_groups_df(all_together, group='orthotopic')

In [ ]:
merged = endog_df.merge(ortho_df, on="names", suffixes=("_endog", "_ortho"))

In [ ]:
from scipy.stats import pearsonr, spearmanr
pearson_corr, _ = pearsonr(merged["scores_endog"], merged["scores_ortho"])
spearman_corr, _ = spearmanr(merged["scores_endog"], merged["scores_endog"])
print(f"Pearson r = {pearson_corr:.2f}, Spearman ρ = {spearman_corr:.2f}")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(6,6))
norm = plt.Normalize(vmin=-10, vmax=10)

# --- Scatter plot ---
scatter = sns.scatterplot(
    data=merged,
    x="scores_endog",
    y="scores_ortho",
    hue="logfoldchanges_endog",
    s=20,
    palette="coolwarm",
    edgecolor="black",
    linewidth=0.1,
    alpha=0.9,
    legend=False
)

# --- Add correlation line (on top) ---
sns.regplot(
    data=merged,
    x="scores_endog",
    y="scores_ortho",
    scatter=False,
    color="black",
    line_kws={"lw": .5, "ls": "--", "alpha": 0.5, "zorder": 10}
)

# Reference lines
plt.axhline(0, color='grey', lw=0.2, ls='--', zorder=1)
plt.axvline(0, color='grey', lw=0.2, ls='--', zorder=1)

# Titles and labels
plt.title(f"Correlation of DE genes\nPearson r = {pearson_corr:.2f}", fontsize=12)
plt.xlabel("Scores (endogenous vs human)", fontsize=11)
plt.ylabel("Scores (orthotopic vs human)", fontsize=11)

# --- Colorbar only ---
sm = plt.cm.ScalarMappable(cmap="coolwarm", norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=plt.gca(), fraction=0.046, pad=0.04)
cbar.set_label("logFC", rotation=270, labelpad=15)

sns.despine()
plt.tight_layout()


plt.savefig(
    "/mnt/kkf2/Cell/AG-Saur/KKF2/Daniele/pdac_atlas_figures/figure6/deg_scores_corr.png",
    dpi=300,
    bbox_inches="tight"
)

plt.show()


### Ortho vs Endo

In [ ]:
sc.tl.rank_genes_groups(mouse_tme, groupby = "Model", layer = "log_norm")

In [ ]:
sc.pl.rank_genes_groups_matrixplot(mouse_all, values_to_plot = "logfoldchanges", cmap = "RdBu_r", min_logfoldchange=2, n_genes=40)

In [ ]:
hallmark_h = dc.op.resource('MSigDB') # hallmark(organism='human')

In [ ]:
net = hallmark_h.rename(columns={'genesymbol' : 'target', 'geneset': 'source'})
net = net.drop_duplicates(subset=['source', 'target'])


In [ ]:
dc.mt.ulm(data=mouse_tme, net=net)

In [ ]:
score = dc.pp.get_obsm(adata=adata, key="score_ulm")

In [ ]:
sc.pl.matrixplot(
    adata=score,
    var_names=score.var_names,
    groupby="Model",
    dendrogram=True,
    standard_scale="var",
    colorbar_title="Z-scaled scores",
    cmap="RdBu_r",
)